In [31]:
pip install quadprog

In [38]:
# Active set method
# for Quadratic programming with nonnegativity constraints
#   min_x  1/2 x^T*Q*x + p'*x
#   s.t.   x >=0
#
# Anh-Huy Phan
# for Convex Optimization and Applications

import numpy as np
from scipy.optimize import linprog, minimize
import quadprog

def gen_example(ex):
  # Define the Q and p matrices and vectors
  if ex == 1:
    Q = np.array([[3, 1, 0, 1],
    [1, 3, 0, 0],
    [0, 0, 4, 1],
    [1, 0, 1, 4]])
    p = np.array([2, -3, 1, 1])
  elif ex == 2:
    n = 10
    Q = np.random.rand(n, n)
    Q = Q @ Q.T # Matrix multiplication
    p = np.random.randn(n)
  elif ex == 3:
    Q = np.array([[3, 1, 1],
    [1, 3, 1],
    [1, 1, 4]])
    p = np.array([1, -3, 2])
  n = len(p)

  Q = Q.astype(np.double)
  p = p.astype(np.double)
  return Q, p

In [39]:
# define the function
def quadprog_activeset(Q, p, x0):
  # min 1/2  x'*Q  x + x'*p
  # s.t.   x >=0
  n = len(p)
  # Initialization
  if len(x0) == 0: #x0 == None:
    x0 = np.maximum(0, np.linalg.solve(-Q, p)) # Solve linear system and element-wise maximum
    #x0 = np.zeros(n)
  x = x0

  # active set S = {i : x(i) = 0}
  S = np.where(np.abs(x) <= 1e-5)[0] # Find the indices where x is zero
  print(f'Active constraints {S}')
  # inactive set : e.g. I = {i : x(i) > 0}
  np.arange(n)

  I = np.setdiff1d(np.arange(n), S) # Find the indices where x is not zero
  print(f'Inactive set {I}')
  maxiters = 100
  stop_flag = False
  for ki in range(maxiters):

    # Estimate a direction vector d
    # which moves x to the new xnew = (x + d) such that
    #    xnew preserves the active set of x,
    # i.e., xnew(S) = 0,
    # implying that d(S) = 0
    #
    # Solve the QP with equality constraints
    #    min   1/2 * (x+d)'*Q*(x+d) + p'(x+d)
    #    s.t.  d(S) == 0
    #
    # or a reduced unconstraint QP for d(I), i.e., inactive set
    #
    #    min 1/2 *d(I)'*Q(I,I)*d(I) + g(I)'*d(I)
    #
    # where g = \nabla f(x) = Q*x + p is the gradient of f
    #
    # S: active set of the current solution
    # I: inactive set of the current solution
    #
    #
    # d has closed-form update
    # d(I) = -Q(I,I)\g(I);
    # Since g = Q * x + p, g(I) = Q(I,I) x(I) + p(I)
    #
    #
    if len(I)==0:
        gS = Q @ x + p # nu

        # check dual feasibility
        # i.e., gradient == dual variables associated with activeset gS(S) > 0
        j = np.where(gS < 0)[0] # Find the indices where gS is negative
        if len(j) > 0:
          # remove the active constraint
          I = np.append(I, S[j[0]])
          S = np.delete(S, j[0])
        else:
          print('Optimal solution found.')
          stop_flag = True

    else:

      d = -x[I] - np.linalg.solve(Q[I, :][:, I], p[I]) # xnew = -Q(I,I)\p(I)

      xnew = x.copy()
      xnew[I] = x[I] + d # -Q(I,I)\p(I)

      # Check dual feasibility
      #
      # - Note that gradient g(i \in I) = 0 ,
      # - We need to check only g(i \in S)
      gS = Q[S, :][:, I] @ xnew[I] + p[S] # nu

      d2 = d.copy()
      d = np.zeros(n)
      d[I] = d2 # note that d(S) = 0

      #  check primal feasibility
      primal_feasibility = np.all(xnew >= -1e-7)

      if primal_feasibility: # When xnew is feasible
        x = xnew
        # check dual feasibility
        # i.e., gradient == dual variables associated with activeset gS(S) > 0
        j = np.where(gS < 0)[0] # Find the indices where gS is negative
        if len(j) > 0:
          # remove the active constraint
          I = np.append(I, S[j[0]])
          S = np.delete(S, j[0])
        else:
          print('Optimal solution found.')
          stop_flag = True

      else: # When xnew is infeasible
        #  Seek the maximum alpha such that
        #  x(t+1) = x(t)+ alpha * d is feasible
        # i.e.
        #     max alpha
        #     s.t x + alpha d >=0
        #
        # Since x(S) = 0, and d(S) =0
        # only need to check for negative d(I)
        #
        #     x(I) + alpha d(I) >=0
        #     alpha <= -x(j)/d(j) % only for d(j)<0  , j in I
        #
        #  alpha = min(-x(j)/d(j)) % d(j)<0  , j in I
        #
        dneg = np.where(d[I] < 0)[0] # Find the indices where d is negative
        alpha = np.min(x[I[dneg]] / (-d[I[dneg]]))

        x = x + alpha * d
        S = np.append(S, I[dneg[np.argmin(x[I[dneg]] / (-d[I[dneg]]))]])
        I = np.setdiff1d(np.arange(n), S)

        #         # a better way is based on LP
        #
        #         I = 1:m;S = [];
        #         alpha = linprog(-1,A*d,b-A*x);
        #         x = x+alpha*d;
        #         ii = find(abs(A*x-b)<=1e-5);
        #         S = [S I(ii)];
        #         I(ii) = [];


    print(f'Run {ki}   |   ', end='')
    print(f'Active constraints {S}  | ', end='')
    print(f'Dual variables {np.sign(gS)}')

    if stop_flag == True:
      #if abs(f(ki)-f(ki-1))<1e-5
      break

  print(f'Active constraints {S}  | ', end='')
  print(f'Dual variables {np.sign(gS)}')
  print(f'Optimal f(x)  = {0.5 * x.T @ Q @ x + p @ x}')
  return x

In [41]:
example_id = 1
[Q,p] = gen_example(example_id)
n = len(p)

# Solve Nonnegative QP using quadprog solver and use it as reference
# Minimize     1/2 x^T G x - a^T x
# Subject to   C.T x >= b
xqp = quadprog.solve_qp(Q, -p, np.eye(n, n), np.zeros(n), meq=0)
# access the first element of the tuple and convert it to an array
x = np.array(xqp[0])
fqp = 0.5 * x.T @ Q @ x + p.T @ x

# Run 1: Solve Nonnegative QP using quadprog_activeset
x0 = np.zeros(n) # Initial point
# Compare solutions obtained by Active set method and SciPy solver
x = quadprog_activeset(Q, p,x0) # Active set method
fx1 =  0.5 * x.T @ Q @ x + p.T @ x # Objective function value

# Run 2: Solve Nonnegative QP using quadprog_activeset with
# Compare solutions obtained by Active set method and SciPy solver
x = quadprog_activeset(Q, p,x0) # Active set method
fx2 =  0.5 * x.T @ Q @ x + p.T @ x # Objective function value

print(f'Optimal QuadProg  f(x)  = {fqp:.3f}')
print(f'Optimal ActiveSet f(x)  = {fx1:.3f}')
print(f'Optimal ActiveSet f(x)  = {fx2:.3f}')


Active constraints [0 1 2 3]
Inactive set []
Run 0   |   Active constraints [0 2 3]  | Dual variables [ 1. -1.  1.  1.]
Optimal solution found.
Run 1   |   Active constraints [0 2 3]  | Dual variables [1. 1. 1.]
Active constraints [0 2 3]  | Dual variables [1. 1. 1.]
Optimal f(x)  = -1.5
Active constraints [0 1 2 3]
Inactive set []
Run 0   |   Active constraints [0 2 3]  | Dual variables [ 1. -1.  1.  1.]
Optimal solution found.
Run 1   |   Active constraints [0 2 3]  | Dual variables [1. 1. 1.]
Active constraints [0 2 3]  | Dual variables [1. 1. 1.]
Optimal f(x)  = -1.5
Optimal QuadProg  f(x)  = -1.500
Optimal ActiveSet f(x)  = -1.500
Optimal ActiveSet f(x)  = -1.500


In [42]:
example_id = 2
[Q,p] = gen_example(example_id)
n = len(p)

# Solve Nonnegative QP using quadprog solver and use it as reference
# Minimize     1/2 x^T G x - a^T x
# Subject to   C.T x >= b
xqp = quadprog.solve_qp(Q, -p, np.eye(n, n), np.zeros(n), meq=0)
# access the first element of the tuple and convert it to an array
x = np.array(xqp[0])
fqp = 0.5 * x.T @ Q @ x + p.T @ x

# Run 1: Solve Nonnegative QP using quadprog_activeset
x0 = np.zeros(n) # Initial point
# Compare solutions obtained by Active set method and SciPy solver
x = quadprog_activeset(Q, p,x0) # Active set method
fx1 =  0.5 * x.T @ Q @ x + p.T @ x # Objective function value

# Run 2: Solve Nonnegative QP using quadprog_activeset with
# Compare solutions obtained by Active set method and SciPy solver
x = quadprog_activeset(Q, p,x0) # Active set method
fx2 =  0.5 * x.T @ Q @ x + p.T @ x # Objective function value

print(f'Optimal QuadProg  f(x)  = {fqp:.3f}')
print(f'Optimal ActiveSet f(x)  = {fx1:.3f}')
print(f'Optimal ActiveSet f(x)  = {fx2:.3f}')

Active constraints [0 1 2 3 4 5 6 7 8 9]
Inactive set []
Run 0   |   Active constraints [0 2 3 4 5 6 7 8 9]  | Dual variables [ 1. -1.  1. -1. -1. -1. -1.  1. -1. -1.]
Run 1   |   Active constraints [0 2 4 5 6 7 8 9]  | Dual variables [ 1.  1. -1.  1. -1. -1.  1. -1. -1.]
Run 2   |   Active constraints [0 2 4 5 6 7 8 9 1]  | Dual variables [ 1.  1.  1. -1. -1.  1. -1. -1.]
Run 3   |   Active constraints [0 2 4 6 7 8 9 1]  | Dual variables [ 1.  1.  1. -1. -1.  1. -1. -1.  1.]
Run 4   |   Active constraints [0 2 4 6 7 8 9 1 3]  | Dual variables [ 1.  1.  1. -1.  1. -1.  1.  1.]
Run 5   |   Active constraints [0 2 4 7 8 9 1 3]  | Dual variables [ 1.  1.  1. -1.  1. -1.  1.  1.  1.]
Run 6   |   Active constraints [0 2 4 7 9 1 3]  | Dual variables [ 1.  1.  1.  1. -1.  1.  1.  1.]
Optimal solution found.
Run 7   |   Active constraints [0 2 4 7 9 1 3]  | Dual variables [1. 1. 1. 1. 1. 1. 1.]
Active constraints [0 2 4 7 9 1 3]  | Dual variables [1. 1. 1. 1. 1. 1. 1.]
Optimal f(x)  = -0.72154

In [43]:
example_id = 3
[Q,p] = gen_example(example_id)
n = len(p)

# Solve Nonnegative QP using quadprog solver and use it as reference
# Minimize     1/2 x^T G x - a^T x
# Subject to   C.T x >= b
xqp = quadprog.solve_qp(Q, -p, np.eye(n, n), np.zeros(n), meq=0)
# access the first element of the tuple and convert it to an array
x = np.array(xqp[0])
fqp = 0.5 * x.T @ Q @ x + p.T @ x

# Run 1: Solve Nonnegative QP using quadprog_activeset
x0 = np.zeros(n) # Initial point
# Compare solutions obtained by Active set method and SciPy solver
x = quadprog_activeset(Q, p,x0) # Active set method
fx1 =  0.5 * x.T @ Q @ x + p.T @ x # Objective function value

# Run 2: Solve Nonnegative QP using quadprog_activeset with
# Compare solutions obtained by Active set method and SciPy solver
x = quadprog_activeset(Q, p,x0) # Active set method
fx2 =  0.5 * x.T @ Q @ x + p.T @ x # Objective function value

print(f'Optimal QuadProg  f(x)  = {fqp:.3f}')
print(f'Optimal ActiveSet f(x)  = {fx1:.3f}')
print(f'Optimal ActiveSet f(x)  = {fx2:.3f}')

Active constraints [0 1 2]
Inactive set []
Run 0   |   Active constraints [0 2]  | Dual variables [ 1. -1.  1.]
Optimal solution found.
Run 1   |   Active constraints [0 2]  | Dual variables [1. 1.]
Active constraints [0 2]  | Dual variables [1. 1.]
Optimal f(x)  = -1.5
Active constraints [0 1 2]
Inactive set []
Run 0   |   Active constraints [0 2]  | Dual variables [ 1. -1.  1.]
Optimal solution found.
Run 1   |   Active constraints [0 2]  | Dual variables [1. 1.]
Active constraints [0 2]  | Dual variables [1. 1.]
Optimal f(x)  = -1.5
Optimal QuadProg  f(x)  = -1.500
Optimal ActiveSet f(x)  = -1.500
Optimal ActiveSet f(x)  = -1.500
